## A Quick Intro to Sonification

First we'll get the python packages we'll need, and then I'll show you a complete script that turns a list of notes into a .mid (midi) file you can hear.

In [ ]:
pip install miditime

In [ ]:
# A demo script that creates two notes
import miditime
from datetime import datetime
from miditime.MIDITime import MIDITime

# Instantiate the class with a tempo (120bpm is the default) and an output file destination.
mymidi = MIDITime(120, 'myfile.mid')

# Create a list of notes. Each note is a list: [time, pitch, attack, duration]
midinotes = [
    [0, 60, 200, 3],  #At 0 beats (the start), Middle C with attack 200, for 3 beats
    [10, 61, 200, 4]  #At 10 beats (12 seconds from start), C#5 with attack 200, for 4 beats
]

# Add a track with those notes
mymidi.add_track(midinotes)

# Output the .mid file
mymidi.save_midi()

Your music file - with just two notes! - is now in your folder for this week. Find the folder using your Finder or Windows Explorer. Double-click it on `myfile.mid`. Whatever program you use for music on your own computer _should_ be able to play it. A mid file is not, in itself, music (in the sense that an mp3 is a compressed representation of the music. It is more akin to the digital representation of a score that the computer plays with a default instrument (often, a piano). Typically, it sounds pretty tinkly.

## Writing music

Play with that script now, and add more notes. Try something simple - the notes for ‘Baa Baa Black Sheep’ are:

D, D, A, A, B, B, B, B, A
Baa, Baa, black, sheep, have, you, any, wool?

Middle C is '60' - [this chart](https://web.archive.org/web/20171211192102/http://www.electronics.dit.ie/staff/tscarff/Music_technology/midi/midi_note_numbers_for_octaves.htm) shows the numerical representation of each note on the 88 key keyboard.


## Representing Data

Let's represent some data

The miditime package is written with time series data in mind:

```json
my_data = [
    {'event_date': <datetime object>, 'magnitude': 3.4},
    {'event_date': <datetime object>, 'magnitude': 3.2},
    {'event_date': <datetime object>, 'magnitude': 3.6},
    {'event_date': <datetime object>, 'magnitude': 3.0},
    {'event_date': <datetime object>, 'magnitude': 5.6},
    {'event_date': <datetime object>, 'magnitude': 4.0}
]```

Here we're dealing with earthquake data, and it's in json notation. The `datetime object` is a particular way of formatting the date: `datetime(1753,6,8)` would be June 8 1753. Following the example that miditime provides, we can build up a sonification of this earthquake data like this:


In [ ]:

# we'll define 'mymidi' which will hold all of our sonified information.
# tempo (120bpm is the default), an output file destination, 
# the number of seconds you want to represent a year in the final song (default is 5 sec/year),
# the base octave (C5 is middle C, so the default is 5, 
# and how many octaves you want your output to range over (default is 1)

mymidi = MIDITime(120, 'second-example.mid', 0.1, 5, 1)



In [ ]:
my_data = [
    {'event_date': datetime(1792,6,8), 'magnitude': 3.4},
    {'event_date': datetime(1800,3,4), 'magnitude': 3.2},
    {'event_date': datetime(1810,1,16), 'magnitude': 3.6},
    {'event_date': datetime(1812,8,23), 'magnitude': 3.0},
    {'event_date': datetime(1813,10,10), 'magnitude': 5.6},
    {'event_date': datetime(1824,1,5), 'magnitude': 4.0}
]



Now we convert those dates into an integer. Oddly enough, this is done by defining time since 'epoch'. This epoch date is Jan 1 1970. The reasons why have to do with the evolution of unix. For dates before 1970, we end up with a negative number, but this is not a problem.

First we convert the date so that it is expressed with reference to the epoch:


In [ ]:
my_data_epoched = [{'days_since_epoch': mymidi.days_since_epoch(d['event_date']), 'magnitude': d['magnitude']} for d in my_data]


In [ ]:
my_data_epoched

And then we convert that integer into something reasonable for music. Per the miditime package:

> Convert your integer date/time to something reasonable for a song. For example, at 120 beats per minute, you'll need to scale the data down a lot to avoid a very long song if your data spans years. This uses the seconds_per_year attribute you set at the top, so if your date is converted to something other than days you may need to do your own conversion. But if your dataset spans years and your dates are in days (with fractions is fine), use the beat() helper method.

In [ ]:
my_data_timed = [{'beat': mymidi.beat(d['days_since_epoch']), 'magnitude': d['magnitude']} for d in my_data_epoched]
my_data_timed

In [ ]:
# Get the earliest date in your series so you can set that to 0 in the MIDI:
start_time = my_data_timed[0]['beat']



Finally, we define how the data get mapped against the pitch. If the data were percentages ranging from 0.01 (ie 1%) to 0.99 (99%), we would scale_pct below between 0 and 1. If you weren’t dealing with percentages, you’d use your lowest value and your highest value (say if for instance your data were counts of pottery over time)

In [ ]:
# Set up some functions to scale your other variable (magnitude in our case) 
# to match your desired mode/key and octave range. 
# There are helper methods to assist this scaling
# You can choose a linear or logarithmic scale.

def mag_to_pitch_tuned(magnitude):
    # Where does this data point sit in the domain of your data? 
    #(I.E. the min magnitude is 3, the max in 5.6). In this case the optional 'True' 
    # means the scale is reversed, so the highest value will return the lowest percentage.
    scale_pct = mymidi.linear_scale_pct(3, 5.7, magnitude)

    # Another option: Linear scale, reverse order
    # scale_pct = mymidi.linear_scale_pct(3, 5.7, magnitude, True)

    # Another option: Logarithmic scale, reverse order
    # scale_pct = mymidi.log_scale_pct(3, 5.7, magnitude, True)

    # Pick a range of notes. This allows you to play in a key.
    c_major = ['C', 'D', 'E', 'F', 'G', 'A', 'B']

    #Find the note that matches your data point
    note = mymidi.scale_to_note(scale_pct, c_major)

    #Translate that note to a MIDI pitch
    midi_pitch = mymidi.note_to_midi_pitch(note)

    return midi_pitch

In [ ]:
# Now build the note list
note_list = []

for d in my_data_timed:
    note_list.append([
        d['beat'] - start_time,
        mag_to_pitch_tuned(d['magnitude']),
        100,  # velocity
        1  # duration, in beats
    ])

# show the result
note_list

In [ ]:
# Add a track with those notes
mymidi.add_track(note_list)

# Output the .mid file
mymidi.save_midi()

# incidentally, to add more tracks, you'd just repeat the process to build more note_lists

Find, then open `second-example.mid` using your computer's music program. How does it sound? It might not be 'music' - but that's not the point.

Try feeding actual archaeological data that you've retrieved from other exercises into this program. Save each result under a different name; you can then begin to mix the data as unique voices using something like GarageBand.

For other approaches to sonification, [please see this tutorial](https://programminghistorian.org/en/lessons/sonification). For another creative use of sonification by an undergraduate, see [Daniel Ruten's Sonic Word Clouds](https://programminghistorian.org/posts/sonic-word-clouds).

Cool, eh?
